In [1]:
import cv2
import numpy as np
import scipy.io
import math
import glob
from skimage import data
from skimage import io
from skimage import color
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from skimage import filters
from scipy import signal
from skimage.filters.rank import gradient
from scipy.ndimage import gaussian_filter
from numpy import linalg as LA
from skimage.transform import pyramid_gaussian
from skimage.morphology import disk
from collections import OrderedDict
from skimage import transform as tf
from scipy.spatial import distance
from random import randint

In [2]:
np.set_printoptions(suppress=True)

# This function takes a list of images as input.
# I loop over the images starting with the second image. I set old_frame to the previous image.
# I set the parameters for the goodFeaturesToTrack function, and the parameters for the optical flow function.
# Then, I convert the old_frame to gray and run it through goodFeaturesToTrack() to get a set of points.
# Then, I do the same with the current frame.
# To get the points for the current frame, I use the calcOpticalFlowPyrLK() by giving it the old and current
# frames and the old points.
# We get back the new points and bool status 'st'. st is 1 if the old point exists in the new frame,
# otherwise, it's 0. I use st to get the good points.
# I then subtract good_old from good_new to get the difference between them (in pixels).
# My thres variable contains the percentage of points that have more than 4 pixels difference
# If the percentage is more than 70% then I count the frame as a shot.
def shot_detector(imgs):
#     det_shots = []
    det_shots = OrderedDict()
    for i in range(1,len(imgs)):

        old_frame = cv2.imread(imgs[i-1])

        # params for ShiTomasi corner detection
        feature_params = dict( maxCorners = 10000,
                               qualityLevel = 0.1,
                               minDistance = 10,
                               blockSize = 5)

        # Parameters for lucas kanade optical flow
        lk_params = dict( winSize  = (9,9),
                          maxLevel = 5,
                          criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 4, 0.2))

        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

        # ------------------------------

        curr_frame = cv2.imread(imgs[i])

        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, curr_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        diff = abs(good_new-good_old)
        thres = (diff > 4).sum()/(len(diff.flatten()))

#         if (int(imgs[i].split('\\')[2][:-4]) > 60 and int(imgs[i].split('\\')[2][:-4]) < 70):
#             print(imgs[i], thres)
#             print('----------------')
        if (thres > 0.7):
#             print(imgs[i], thres)
            # converting the image name to just the number of the image by splitting \, taking the 3rd part, and removing the extension
            s = int(imgs[i].split('\\')[2][:-4])
            if (len(det_shots) and list(det_shots.keys())[-1] == s-1 and list(det_shots.values())[-1] < thres):
                det_shots.pop(list(det_shots.keys())[-1])
            det_shots[s] = thres
            
        det_shots_list = list(det_shots.keys())
#     print(det_shots)
#     print(len(det_shots_list))
    return det_shots_list

In [11]:
# This function takes detected shots and number of the clip as input.
# Depending on the number of the clip, I set the annot_shots list to the cooresponding list of
# annotated shots. I also set the total to the number of shots in the list.
# Then, I loop over the shots in the annotated list and check if it exists in the detected shots list.
# If it is, I add 1 to the score and remove the shot from the detected shots list.
# If the shots is off by 1 frame, I consider it correct and add 1 to score and remove the shot from the 
# detected shots list. I remove the shot so that I can count the remaining the shots as false positives.
# At the end, I subtract the false positives from the score and print the score.

def shot_performance(det_shots, clip_num):
    annot_shots = []
    score = 0
    total = 0
    clip = ''
    
    if (clip_num == 1):
        annot_shots = ["156"]
#         score = len(annot_shots)
        total = len(annot_shots)
        clip = 'clip 1'

    elif (clip_num == 2):
        annot_shots = ["066","122","139","144","152","164","178","189"]
#         score = len(annot_shots)
        total = len(annot_shots)
        clip = 'clip 2'
        
    elif (clip_num == 3):
        annot_shots = ["0051","0057","0059","0061","0063","0065","0067",
                       "0069","0071","0072","0074","0076","0078","0080",
                       "0082","0084","0086","0087","0091","0093","0096",
                       "0104","0165","0187","0262","0267"]

        total = len(annot_shots)
        clip = 'clip 3'
        
    for shot in annot_shots:
        if (int(shot) in det_shots):
            score += 1
            det_shots.remove(int(shot))
            
        elif (int(shot)-1 in det_shots):
            score += 1
            det_shots.remove(int(shot)-1)
            
        elif (int(shot)+1 in det_shots):
            score += 1
            det_shots.remove(int(shot)+1)
            
#     print(score, len(det_shots))
    
    false_pos = len(det_shots)
    print(clip)
    print('Correct hits: ', score, '/', len(annot_shots))
    print('False hits: ', false_pos)

    score = score - (false_pos/2)
    
    if (score < 0):
        print('Too many missed and/or false positives')
    
    else:
        score = (score/total) * 100
    
    print('Score: ', score,'/ 100')
#     return score


In [12]:
clip1 = glob.glob('.\\clip_1/*.jpg')
clip2 = glob.glob('.\\clip_2/*.jpg')
clip3 = glob.glob('.\\clip_3/*.jpg')

shots1 = shot_detector(clip1)
# print(shots1)
shot_performance(shots1, 1)
print('----------------')
shots2 = shot_detector(clip2)
# print(shots2)
shot_performance(shots2, 2)
print('----------------')
shots3 = shot_detector(clip3)
# print(shots3)
shot_performance(shots3, 3)

clip 1
Correct hits:  1 / 1
False hits:  0
Score:  100.0 / 100
----------------
clip 2
Correct hits:  7 / 8
False hits:  1
Score:  81.25 / 100
----------------
clip 3
Correct hits:  17 / 26
False hits:  6
Score:  53.84615384615385 / 100


In [7]:
#"0051.jpg","0057.jpg","0059.jpg","0061.jpg","0063.jpg","0065.jpg","0067.jpg","0069.jpg","0071.jpg","0072.jpg","0074.jpg","0076.jpg","0078.jpg","0080.jpg","0082.jpg","0084.jpg","0086.jpg","0087.jpg","0091.jpg","0093.jpg","0096.jpg","0104.jpg","0165.jpg","0187.jpg","0262.jpg","0267.jpg",
#"066.jpg","122.jpg","139.jpg","144.jpg","152.jpg","164.jpg","178.jpg","189.jpg"